In [1]:
# Apply custon style to notebook
from IPython.core.display import HTML
import pathlib
styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
styles = open(styles_path, "r").read()
HTML(f"<style>{styles}</style>")

_Note: This notebook must be run locally. Due to how the Swift Simulator operates, this notebook will not run on Google Colab_

# 5.0 Quadratic Programming for Motion Control

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[5.1 Swift and Robot Setup](#swift)

[5.2 Basic Quadratic Programming](#bqp)
* Advanced Velocity Control
  * Null-space Projection
  * Quadratic Programming

[5.3 Velocity Dampers](#vd)
* Advanced Velocity Control
  * Null-space Projection
  * Quadratic Programming

[5.4 Adding Slack](#slack)
* Advanced Velocity Control
  * Null-space Projection
  * Quadratic Programming

In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# swift is a lightweight browser-based simulator which comes eith the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# this package provides several solvers for solving quadratic programmes
import qpsolvers as qp

<br>

<a id='swift'></a>

### 5.1 Swift and Robot Setup
---

In this notebook we will be using the Robotics Toolbox for Python and Swift to simulate our motion controllers. Check out Part 1 Notebook 3 for an introduction to these packages.

In [3]:
# Make the environment
env = Swift()

# Launch the simulator
# The realtime flag will ask the simulator to simulate as close as
# possible to realtime as apposed to as fast as possible
env.launch(realtime=True, browser="notebook")

# Make a panda robot
panda = rtb.models.Panda()

# Set the joint coordinates to qr
panda.q = panda.qr

# We can then add our robot to the simulator envionment
env.add(panda)

# end-effector axes
ee_axes = sg.Axes(0.1)

# goal axes
goal_axes = sg.Axes(0.1)

# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes) 

2

In [4]:
# Make a variable for the upper and lower limits of the robot
qd_lb = -20.0 * np.ones(panda.n)
qd_ub = 20.0 * np.ones(panda.n)

<br>

<a id='bqp'></a>

### 4.2 Basic Quadratic Programming
---

In this Notebook, we are going redefine our motion controllers as a quadratic programming (QP) optimisation problem rather than a matrix equation. In general, a constrained QP is formulated as

\begin{align*} 
    \min_x \quad f_o(\bf{x}) &= \frac{1}{2} \bf{x}^\top \bf{Q} \bf{x}+ \bf{c}^\top \bf{x}, \\ 
    \text{subject to} \quad \bf{A}_1 \bf{x} &= \bf{b}_1,  \\
    \bf{A}_2 \bf{x} &\leq \bf{b}_2,  \\
    \bf{g} &\leq  \bf{x} \leq \bf{h}.  
\end{align*}

where $f_o(\bf{x})$ is the objective function which is subject to the following equality and inequality constraints, and $\bf{h}$ and $\bf{g}$ represent the upper and lower bounds of $\bf{x}$. A quadratic program is strictly convex when the matrix $\bf{Q}$ is positive definite. This framework allows us to solve the same problems as above, but with additional flexibility.

We can rewrite RRMC (see Part 1 Notebook 3) as a QP

\begin{align*} 
	\min_{\dot{\bf{q}}} \quad f_o(\dot{\bf{q}}) 
	&= 
    \frac{1}{2} 
    \dot{\bf{q}}^\top 
    \bf{1}_n 
    \dot{\bf{q}}, \\
	\text{subject to} \quad
	\bf{J}(\bf{q}) \dot{\bf{q}} &= \bf{\nu},
	 \\
	\dot{\bf{q}}^- &\leq \dot{\bf{q}} \leq \dot{\bf{q}}^+  
\end{align*}

where $\bf{1}_n \in \mathbb{R}^{n \times n}$ is an identity matrix, and we have imposed $\dot{\bf{q}}^-$ and $\dot{\bf{q}}^+$ as upper and lower joint-velocity limits. If the robot has more degrees-of-freedom than necessary to reach its entire task space, the QP will achieve the desired end-effector velocity with the minimum joint-velocity norm (the same result as the psuedoinverse solution).

Lets test this out in Python

In [5]:
# Make a new environment and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.set_camera_pose([1.3, 0, 0.4], [0, 0, 0.3])
env.add(panda)
env.add(ee_axes)
env.add(goal_axes)

# Change the robot configuration to a ready position
panda.q = [0.21, -0.03, 0.35, -1.90, -0.04, 1.96, 1.36]

# A variable to specify when to break the loop
arrived = False

# Specify the gain for the p_servo method
kt = 1.0
kr = 1.3
k = np.array([kt, kt, kt, kr, kr, kr])

# Specify our timestep
dt = 0.05

Tep = (
    panda.fkine(panda.q)
    * sm.SE3.Tx(-0.1)
    * sm.SE3.Ty(0.6)
    * sm.SE3.Tz(0.4)
)
Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# The quadratic component of the objective function
Q = np.eye(panda.n)

# Linear component of objective function
c = np.ones(panda.n)

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda.jacob0(panda.q)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda.fkine(panda.q).A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    ### Calculate each component of the quadratic programme
    # The equality contraints
    Aeq = J
    beq = ev.reshape((6,))

    # The inequality constraints
    Ain = None
    bin = None

    # The lower and upper bounds on the joint velocity
    lb = qd_lb
    ub = qd_ub

    # Solve for the joint velocities qd and apply to the robot
    panda.qd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver='quadprog')

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

We can rewrite Park's controller (see Part 2 Notebook 4) as

\begin{align*} 
    \min_{\dot{\bf{q}}} \quad f_o(\dot{\bf{q}}) 
    &= 
    \frac{1}
        {2} 
    \dot{\bf{q}}^\top \lambda_q \bf{1}_n \dot{\bf{q}} - \lambda_m \ \bf{J}_m^\top(\bf{q}) \dot{\bf{q}}, \\ 
    \text{subject to} \quad \bf{J}(\bf{q}) \dot{\bf{q}} &= \bf{\nu},
	 \\
	\dot{\bf{q}}^- &\leq \dot{\bf{q}} \leq \dot{\bf{q}}^+  
\end{align*}

where the manipulability Jacobian fits into the linear component of the objective function.

Lets implement this below

In [6]:
# Make a new environment and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.set_camera_pose([1.3, 0, 0.4], [0, 0, 0.3])
env.add(panda)
env.add(ee_axes)
env.add(goal_axes) 

# Change the robot configuration to a ready position
panda.q = [0.21, -0.03, 0.35, -1.90, -0.04, 1.96, 1.36]

# A variable to specify when to break the loop
arrived = False

# Specify the gain for the p_servo method
kt = 1.0
kr = 1.3
k = np.array([kt, kt, kt, kr, kr, kr])

# Specify our timestep
dt = 0.05

Tep = (
    panda.fkine(panda.q)
    * sm.SE3.Tx(-0.1)
    * sm.SE3.Ty(0.6)
    * sm.SE3.Tz(0.4)
)
Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# Set the gain on the manipulability maximisation
λm = 10.0

# Set the gain on the joint velocity norm minimisation
λq = 1.0

Q = λq * np.eye(panda.n)

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda.jacob0(panda.q)

    # Calculate the manipulability Jacobian
    Jm = panda.jacobm(panda.q, axes='rot')

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda.fkine(panda.q).A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    ### Calculate each component of the quadratic programme
    # The equality contraints
    Aeq = J
    beq = ev.reshape((6,))

    # The inequality constraints
    Ain = None
    bin = None

    # Linear component of objective function: the manipulability Jacobian
    c = λm * -Jm.reshape((panda.n,))

    # The lower and upper bounds on the joint velocity
    lb = qd_lb
    ub = qd_ub

    # Solve for the joint velocities qd and apply to the robot
    panda.qd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver='quadprog')

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

<br>

<a id='vd'></a>

### 5.3 Velocity Dampers
---

We can expand the above with velocity dampers to enable joint-position limit avoidance. Velocity dampers are used to constrain velocities and dampen an input velocity as some limit is approached. The velocity is only damped if it is within some influence distance to the limit. A joint velocity is constrained to prevent joint limits using a velocity damper as

\begin{align*} 
    \dot{q} \leq
    \eta
    \frac{\rho - \rho_s}
        {\rho_i - \rho_s}
    \qquad \text{if} \ \rho < \rho_i
\end{align*}

where $\rho \in \mathbb{R}^+$ is the distance or angle to the nearest joint limit, $\eta \in \mathbb{R}^+$ is a gain which adjusts the aggressiveness of the damper, $\rho_i$ is the influence distance in which to activate the damper, and $\rho_s$ is the stopping distance in which the distance $\rho$ will never be able to reach or enter. We can stack velocity dampers to perform joint position limit avoidance for each joint within a robot, and incorporate it into our QP as an inequality constraint

\begin{align*} 
    \bf{1}_n
    \dot{\bf{q}} &\leq
    \eta
    \begin{pmatrix}
        \frac{\rho_0 - \rho_s}
            {\rho_i - \rho_s} \\
        \vdots \\
        \frac{\rho_n - \rho_s}
            {\rho_i - \rho_s} 
    \end{pmatrix}
\end{align*}

where the identity $\bf{1}_n$ is included to show how the equation fits into the general form $\bf{A}\bf{x} \leq \bf{b}$ of an inequality constraint.

Lets make a Python method to construct these joint position limit velocity dampers

In [7]:
def joint_velocity_damper(robot, ps, pi, gain):
    """
    Formulates an inequality contraint which, when optimised for will
    make it impossible for the robot to run into joint limits. Requires
    the joint limits of the robot to be specified.

    ps: The minimum angle (in radians) in which the joint is
        allowed to approach to its limit
    pi: The influence angle (in radians) in which the velocity
        damper becomes active
    gain: The gain for the velocity damper

    returns: Ain, Bin as the inequality contraints for an qp
    """

    Ain = np.zeros((robot.n, robot.n))
    Bin = np.zeros(robot.n)

    for i in range(robot.n):
        if robot.q[i] - robot.qlim[0, i] <= pi:
            Bin[i] = -gain * (((robot.qlim[0, i] - robot.q[i]) + ps) / (pi - ps))
            Ain[i, i] = -1
        if robot.qlim[1, i] - robot.q[i] <= pi:
            Bin[i] = gain * ((robot.qlim[1, i] - robot.q[i]) - ps) / (pi - ps)
            Ain[i, i] = 1

    return Ain, Bin

<br>

<a id='slack'></a>

### 5.4 Adding Slack
---

It is possible that the robot will fail to reach the goal when the constraints create local minima. In such a scenario, the error $\bf{e}$ in the PBS scheme is no longer decreasing and the robot can no longer progress due to the constraints in the QP.

The methods shown up to this point require a redundant robot, where the number of joints is greater than 6. In [8], extra redundancy was introduced to the QP by relaxing the equality constraint in the QP to allow for intentional deviation from the desired end-effector velocity $\bf{\nu}$. The slack has the additional benefit of providing the solver extra flexibility to meet constraints and avoid local minima. The augmented QP is defined as

\begin{align*} 
    \min_x \quad f_o(\bf{x}) &= \frac{1}{2} \bf{x}^\top \mathcal{Q} \bf{x}+ \mathcal{C}^\top \bf{x}, \\ 
    \text{subject to} \quad \mathcal{J} \bf{x} &= \bf{\nu},  \\
    \mathcal{A} \bf{x} &\leq \mathcal{B},  \\
    \bf{x}^- &\leq \bf{x} \leq \bf{x}^+ 
\end{align*}

with

\begin{align*}
    \bf{x} &= 
    \begin{pmatrix}
        \dot{\bf{q}} \\ \bf{\delta}
    \end{pmatrix} \in \mathbb{R}^{(n+6)}  \\
    \mathcal{Q} &=
    \begin{pmatrix}
        \lambda_q \bf{1}_{n} & \mathbf{0}_{6 \times 6} \\ \mathbf{0}_{n \times n} & \lambda_\delta \bf{1}_{6}
    \end{pmatrix} \in \mathbb{R}^{(n+6) \times (n+6)} \\
    \mathcal{J} &=
    \begin{pmatrix}
        \bf{J}(\bf{q}) & \bf{1}_{6}
    \end{pmatrix} \in \mathbb{R}^{6 \times (n+6)} \\
    \mathcal{C} &= 
    \begin{pmatrix}
        \bf{J}_m \\ \bf{0}_{6 \times 1}
    \end{pmatrix} \in \mathbb{R}^{(n + 6)} \\
    \mathcal{A} &= 
    \begin{pmatrix}
        \bf{1}_{n \times n + 6} \\
    \end{pmatrix} \in \mathbb{R}^{(l + n) \times (n + 6)} \\
    \mathcal{B} &= 
    \eta
    \begin{pmatrix}
        \frac{\rho_0 - \rho_s}
                {\rho_i - \rho_s} \\
        \vdots \\
        \frac{\rho_n - \rho_s}
                {\rho_i - \rho_s} 
    \end{pmatrix} \in \mathbb{R}^{n} \\
    \bf{x}^{-, +} &= 
    \begin{pmatrix}
        \dot{\bf{q}}^{-, +} \\
        \bf{\delta}^{-, +}
    \end{pmatrix} \in \mathbb{R}^{(n+6)},
\end{align*}

where
$\bf{\delta} \in \mathbb{R}^6$ is the slack vector,
$\lambda_\delta \in \mathbb{R}^+$ is a gain term which adjusts the cost of the norm of the slack vector in the optimiser,
$\dot{\bf{q}}^{-,+}$ are the minimum and maximum joint velocities, and 
$\dot{\bf{\delta}}^{-,+}$ are the minimum and maximum slack velocities.
Each of the gains can be adjusted dynamically. For example, in practice $\lambda_\delta$ is typically large when far from the goal, but reduces towards 0 as the goal approaches.

The effect of this augmented optimisation problem is that the equality constraint is equivalent to

\begin{align*} 
    \bf{\nu}(t) - \bf{\delta}(t) = \bf{J}(\bf{q}) \dot{\bf{q}}(t)
\end{align*}

which clearly demonstrates that the slack is essentially intentional error, where the optimiser can choose to move components of the desired end-effector motion into the slack vector. For both redundant and non-redundant robots, this means that the robot may stray from the straight line motion to improve manipulability and avoid a singularity, avoid running into joint position limits, or stay bounded by the joint velocity limits.

Lets put this algorithm to the test in Python

In [8]:
# Make a new environment and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.set_camera_pose([1.3, 0, 0.4], [0, 0, 0.3])
env.add(panda)
env.add(ee_axes)
env.add(goal_axes) 

# Change the robot configuration to a ready position
panda.q = [0.21, -0.03, 0.35, -1.90, -0.04, 1.96, 1.36]

# A variable to specify when to break the loop
arrived = False

# Specify the gain for the p_servo method
kt = 1.0
kr = 1.3
k = np.array([kt, kt, kt, kr, kr, kr])

# Specify our timestep
dt = 0.05

Tep = (
    panda.fkine(panda.q)
    * sm.SE3.Tx(-0.1)
    * sm.SE3.Ty(0.6)
    * sm.SE3.Tz(0.4)
)
Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# Set the gain on the manipulability maximisation
λm = 1.0

# Set the gain on the joint velocity norm minimisation
λq = 0.1

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda.jacob0(panda.q)

    # Calculate the manipulability Jacobian
    Jm = panda.jacobm(panda.q, axes='rot')

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda.fkine(panda.q).A

    # Spatial error
    e = np.sum(np.abs(rtb.angle_axis(Te, Tep)))

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    ### Calculate each component of the quadratic programme
    # Quadratic component of objective function
    Q = np.eye(panda.n + 6)

    # Joint velocity component of Q
    Q[:panda.n, :panda.n] *= λq

    # Slack component of Q
    Q[panda.n:, panda.n:] = (1 / e) * np.eye(6)

    # The equality contraints
    Aeq = np.c_[J, np.eye(6)]
    beq = ev.reshape((6,))

    # The inequality constraints for joint limit avoidance
    Ain = np.zeros((panda.n + 6, panda.n + 6))
    bin = np.zeros(panda.n + 6)

    # The minimum angle (in radians) in which the joint is allowed to approach
    # to its limit
    ps = 0.05

    # The influence angle (in radians) in which the velocity damper
    # becomes active
    pi = 0.9

    # The gain for the velocity damper
    k_qlim = 1.0

    # Form the joint limit velocity damper
    Ain[:panda.n, :panda.n], bin[:panda.n] = joint_velocity_damper(panda, ps, pi, k_qlim)

    # Linear component of objective function: the manipulability Jacobian
    c = np.r_[λm * -Jm.reshape((panda.n,)), np.zeros(6)]

    # The lower and upper bounds on the joint velocity and slack variable
    lb = np.r_[qd_lb, -10 * np.ones(6)]
    ub = np.r_[qd_ub, 10 * np.ones(6)]

    # Solve for the joint velocities qd and apply to the robot
    xd = qp.solve_qp(Q, c, Ain, bin, Aeq, beq, lb=lb, ub=ub, solver='quadprog')

    # Apply the joint velocities to the Panda
    panda.qd[:panda.n] = xd[:panda.n]

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)